In [1]:
import os
os.chdir("../")

In [2]:
import glob
import pandas as pd
import time
import numpy as np
from scipy import stats

In [3]:
resultDir = 'results'
problem = 'setcover' # choices=['setcover', 'cauctions', 'facilities', 'indset']
problemSizes = ['small', 'medium', 'big']
sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]

In [4]:
# f1 = "cauctions_20210710-004536.csv"
# f2 = "cauctions_20210710-095340.csv"
# df1 = pd.read_csv(f"{resultDir}/{f1}")
# df2 = pd.read_csv(f"{resultDir}/{f2}")
# df = pd.concat([df1, df2], ignore_index=True)
## df.to_csv(f'{resultDir}/eval_cauctions_ss2ts21_25_dsSmall_trainFast.csv')

df = pd.read_csv(f'{resultDir}/cauctions_20210810-103421.csv')

In [5]:
# f1 = "setcover_20210725-205924.csv"
# f2 = "setcover_20210726-163252.csv"
# df1 = pd.read_csv(f"{resultDir}/{f1}")
# df2 = pd.read_csv(f"{resultDir}/{f2}")
# df = pd.concat([df1, df2], ignore_index=True)
# df.to_csv(f'{resultDir}/eval_cauctions_ss2ts21_25_dsSmall_trainFast.csv')

In [5]:
# eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
# eval_file = eval_files[-1]
# eval_file = f"{resultDir}/cauctions_20210710-004536.csv"

# df = pd.read_csv(eval_file)
# df_list = []
# [df[df['type'] == probSize] for probSize in problemSizes]

# df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = pd.concat([df[df['type'] == probSize] for probSize in problemSizes])
df = df.astype({'nlps': float, 'nnodes' : float})

df_gcnns = df[df['policy'] != 'internal:relpscost'].copy()

In [6]:
def gmean_1shifted(x):
    return stats.mstats.gmean(x + 1) - 1

# 1. Means

In [7]:
dfgcnns_gmean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].agg(gmean_1shifted)
dfgcnns_mean = df_gcnns.groupby(['type','sampling_strategy'])[['nnodes', 'stime']].mean()
dfgcnns_gmean

nnodes       stime
type   sampling_strategy                          
big    depthK             13907.427752  186.114510
       depthK2            14989.880418  201.799121
       uniform5           16387.536895  216.086380
medium depthK              1181.989865   16.571325
       depthK2             1210.703225   16.990004
       uniform5            1267.703925   17.639268
small  depthK                65.934661    1.738163
       depthK2               67.619984    1.770530
       uniform5              66.588782    1.766158

In [8]:
def norm_by_uniform5(dfgcnns_grouped):
    df_list = []
    for probSize in dfgcnns_grouped.index.levels[0]:
        df_list.append(dfgcnns_grouped.loc[probSize] / dfgcnns_grouped.loc[(probSize, 'uniform5')])
    return pd.concat(df_list, keys=dfgcnns_grouped.index.levels[0])

In [9]:
dfgcnns_gmean_norm = norm_by_uniform5(dfgcnns_gmean)
dfgcnns_mean_norm = norm_by_uniform5(dfgcnns_mean)
dfgcnns_gmean_norm

nnodes     stime
type   sampling_strategy                    
big    depthK             0.848659  0.861297
       depthK2            0.914712  0.933882
       uniform5           1.000000  1.000000
medium depthK             0.932386  0.939457
       depthK2            0.955036  0.963192
       uniform5           1.000000  1.000000
small  depthK             0.990177  0.984149
       depthK2            1.015486  1.002475
       uniform5           1.000000  1.000000

# 2. Std variances (per instance)

In [10]:
dfgcnns_std_norm_perInstance = df_gcnns.groupby(['type','sampling_strategy','instance']).std() / df_gcnns.groupby(['type','sampling_strategy','instance']).mean()

In [11]:
dfgcnns_std_norm = dfgcnns_std_norm_perInstance.groupby(['type','sampling_strategy'])[['nnodes','stime']].mean()
dfgcnns_std_norm

nnodes     stime
type   sampling_strategy                    
big    depthK             0.128162  0.113212
       depthK2            0.121177  0.107726
       uniform5           0.366205  0.309037
medium depthK             0.119783  0.087547
       depthK2            0.122792  0.099680
       uniform5           0.171346  0.135637
small  depthK             0.157939  0.093488
       depthK2            0.141575  0.087161
       uniform5           0.158202  0.091156

# 做一下 T-Test

# 计算Wins

In [12]:
ttest_res = pd.DataFrame(index=dfgcnns_mean_norm.index, columns=pd.MultiIndex.from_product((['nnodes', 'stime'], ['t_stats', 'p_value'])))
for metric in ['nnodes', 'stime']:
    for probSize in dfgcnns_mean_norm.index.levels[0]: # medium, small
        mean2 = 1
        std2 = dfgcnns_std_norm.at[(probSize,'uniform5'),metric]
        for sampling_strategy in dfgcnns_mean_norm.index.levels[1]: # depthK, depthK2, uniform5
            mean1 = dfgcnns_mean_norm.at[(probSize,sampling_strategy),metric]
            std1_norm = dfgcnns_std_norm.at[(probSize,sampling_strategy),metric]
            std1 = std1_norm * mean1
            t_stats, p_value = stats.ttest_ind_from_stats(mean1, std1, 100, mean2, std2, 100)
            ttest_res.loc[(probSize,sampling_strategy),(metric,)] = [t_stats, p_value]

In [13]:
ttest_res

nnodes               stime          
                           t_stats   p_value   t_stats   p_value
type   sampling_strategy                                        
big    depthK            -6.985106       0.0 -7.381872       0.0
       depthK2           -4.011731  0.000085 -3.563597  0.000458
       uniform5                0.0       1.0       0.0       1.0
medium depthK            -5.630129       0.0 -5.928913       0.0
       depthK2           -2.944054  0.003627 -2.909434  0.004035
       uniform5                0.0       1.0       0.0       1.0
small  depthK            -0.979913  0.328325 -1.478786  0.140787
       depthK2            1.220814  0.223608  0.346274  0.729504
       uniform5                0.0       1.0       0.0       1.0

In [14]:
winner_idx = df_gcnns.groupby(['type','instance','seed'])['stime'].agg(pd.Series.idxmin)
winner_idx

type   instance                                                  seed
big    data/instances/cauctions/transfer_300_1500/instance_1.lp  0       605.0
                                                                 1       606.0
                                                                 2       607.0
                                                                 3       608.0
                                                                 4       609.0
                                                                         ...  
small  data/instances/cauctions/transfer_100_500/instance_9.lp   0       125.0
                                                                 1       131.0
                                                                 2       122.0
                                                                 3       128.0
                                                                 4       129.0
Name: stime, Length: 300, dtype: float64

In [15]:
df_gcnns['wins'] = 0
df_gcnns.loc[winner_idx,'wins'] = 1

In [16]:
len(winner_idx)

300

In [17]:
dfgcnns_wins = df_gcnns.groupby(['type', 'sampling_strategy'])['wins'].sum()
dfgcnns_wins

type    sampling_strategy
big     depthK               51
        depthK2              23
        uniform5             26
medium  depthK               49
        depthK2              29
        uniform5             22
small   depthK               40
        depthK2              21
        uniform5             39
Name: wins, dtype: int64

In [18]:
stats.ttest_ind_from_stats(0.967777, 0.085076*0.967777, 100, 1, 0.102216, 100)

Ttest_indResult(statistic=-2.455047672770633, pvalue=0.014949535302005438)

# 把Means、std 和 Wins合并起来

In [19]:
pd.concat([dfgcnns_gmean_norm, dfgcnns_std_norm, dfgcnns_wins], axis=1, keys=['gmean','std','wins'])

gmean                 std           wins
                            nnodes     stime    nnodes     stime wins
type   sampling_strategy                                             
big    depthK             0.848659  0.861297  0.128162  0.113212   51
       depthK2            0.914712  0.933882  0.121177  0.107726   23
       uniform5           1.000000  1.000000  0.366205  0.309037   26
medium depthK             0.932386  0.939457  0.119783  0.087547   49
       depthK2            0.955036  0.963192  0.122792  0.099680   29
       uniform5           1.000000  1.000000  0.171346  0.135637   22
small  depthK             0.990177  0.984149  0.157939  0.093488   40
       depthK2            1.015486  1.002475  0.141575  0.087161   21
       uniform5           1.000000  1.000000  0.158202  0.091156   39

In [20]:
# output_idx = pd.MultiIndex.from_product((dfgcnns_gmean_norm.index.get_level_values('type').unique(), sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx = pd.MultiIndex.from_product((problemSizes, sampling_Strategies), names=['type', 'sampling_strategy'])
output_idx

MultiIndex([( 'small', 'uniform5'),
            ( 'small',   'depthK'),
            ( 'small',  'depthK2'),
            ('medium', 'uniform5'),
            ('medium',   'depthK'),
            ('medium',  'depthK2'),
            (   'big', 'uniform5'),
            (   'big',   'depthK'),
            (   'big',  'depthK2')],
           names=['type', 'sampling_strategy'])

In [21]:
output_df_mean = dfgcnns_gmean_norm.reindex(output_idx)
output_df_std = dfgcnns_std_norm.reindex(output_idx)
output_df_wins = dfgcnns_wins.reindex(output_idx)
output_df_ttest = ttest_res.reindex(output_idx)

In [22]:
out_df1 = pd.DataFrame(index=output_idx, columns=['nnodes', 'stime', 'wins', 't-stats (p-val)'])
out_df1

nnodes stime wins t-stats (p-val)
type   sampling_strategy                                  
small  uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
       depthK2              NaN   NaN  NaN             NaN
medium uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
       depthK2              NaN   NaN  NaN             NaN
big    uniform5             NaN   NaN  NaN             NaN
       depthK               NaN   NaN  NaN             NaN
       depthK2              NaN   NaN  NaN             NaN

In [23]:
for col_name in ['nnodes', 'stime']:
    out_df1[col_name] = ["%.4fr ± %.2f" % (m, s*100) + "%" for (m, s) in zip(output_df_mean[col_name], output_df_std[col_name])]
out_df1['wins'] = output_df_wins

In [24]:
out_df1['t-stats (p-val)'] = ["%.2f(%.4f)" % (t, p) for (t, p) in zip(output_df_ttest[('stime','t_stats')], output_df_ttest[('stime','p_value')])]

In [25]:
out_df1

nnodes             stime  wins  \
type   sampling_strategy                                             
small  uniform5           1.0000r ± 15.82%   1.0000r ± 9.12%    39   
       depthK             0.9902r ± 15.79%   0.9841r ± 9.35%    40   
       depthK2            1.0155r ± 14.16%   1.0025r ± 8.72%    21   
medium uniform5           1.0000r ± 17.13%  1.0000r ± 13.56%    22   
       depthK             0.9324r ± 11.98%   0.9395r ± 8.75%    49   
       depthK2            0.9550r ± 12.28%   0.9632r ± 9.97%    29   
big    uniform5           1.0000r ± 36.62%  1.0000r ± 30.90%    26   
       depthK             0.8487r ± 12.82%  0.8613r ± 11.32%    51   
       depthK2            0.9147r ± 12.12%  0.9339r ± 10.77%    23   

                         t-stats (p-val)  
type   sampling_strategy                  
small  uniform5             0.00(1.0000)  
       depthK              -1.48(0.1408)  
       depthK2              0.35(0.7295)  
medium uniform5             0.00(1.0000)  
       depthK              -5.93(0.0000)  
       depthK2             -2.91(0.0040)  
big    uniform5             0.00(1.0000)  
       depthK              -7.38(0.0000)  
       depthK2             -3.56(0.0005)